# Lab 1: Classification! (and some n-gram math)

#### Due on 02/02/2024 @ 21:00 hours

Agenda
------
+ Detecting the end of a sentence
    - Rule-based classifier
+ Detecting the sentiment of a sentence
    - Rule-based classifier (counting words)
    - Measuring Accuracy, Precision, Recall (evaluating a classifier)
+ N-gram Math (getting started on things for HW 3)


Looking ahead, we'll be focusing on *classification* for much of the next several weeks. Classification can take several forms. Here are some vocabulary terms to get you started:

- __classifier__: a model that takes data (text, in NLP) as input and outputs a category
- __binary classification__: a model that takes input and outputs *one of two* categories (e.g. "positive" or "negative")
- __multinomial classification__: a model that takes input and outputs *one of many* categories (e.g. "positive", "neutral" or "negative" or a language model that chooses one token from the entire vocabulary)


- __rule-based classifier__: a classifier that functions based on rules that humans come up with (e.g. "the end of a sentence is when there is a "." ")
- __statistical classifier__: a classifier that functions based on counts (statistics) that it has gathered or based on running an algorithm to automatically train parameters on a given data set. 
    
In this lab, you'll be building rule-based classifiers and evaluating them. We'll learn about our first statistical classifier next lecture

All tasks have equal weight.

# Task 0: Who is in your group?

__Ryan Tietjen__

# Task 1: Detecting the end of a sentence


A classifier is, in essence, a function that takes some data $x$ and assigns some label $y$ to it. For a binary classifier, we can model this a function that takes a data point $x$ and returns either `True` or `False`.

Later in this class we'll learn about how to build classifiers that automatically learn how to do this, but we'll start where NLP started—writing some rule-based classifiers.

In [1]:
def classify_sentence_end(text: str, target_index: int) -> bool: 
    """
    Classify whether or not a *location* is the end of a sentence within
    a given text
    Parameters:
        text - string piece of text
        target_index - int candidate location
    returns true if the target index is the end of a sentence. 
    False otherwise. 
    """
    if target_index == len(text) - 1:
        return True
    elif text[target_index] == "." and text[target_index+1] == " ":
        return True
    return False

# look at the code in the cell below to see example usage

In [2]:
# example text
# feel free to go through different examples

# This is the given example text
"""Stocks were up as advancing issues outpaced declining issues 
          on the NYSE by 1.5 to 1. Large- and small-cap stocks were both strong, 
          while the S.&P. 500 index gained 0.46% to finish at 2,457.59. Among 
          individual stocks, the two top percentage gainers in the S.&P. 500 
          were Incyte Corporation and Gilead Sciences Inc."""

example = "Stocks were up as advancing issues outpaced declining issues on the NYSE by 1.5 to 1. Large- and small-cap stocks were both strong, while the S.&P. 500 index gained 0.46% to finish at 2,457.59. Among individual stocks, the two top percentage gainers in the S.&P. 500 were Incyte Corporation and Gilead Sciences Inc."

# this code will go through and
# build up a string based on the sentence
# decisions that your classifier comes up with
# it will put "****" between the sentences
# you do not need to modify any code here
so_far = ""
for index in range(len(example)):
    # see how the classify_sentence_end function is called!
    result = classify_sentence_end(example, index)
    so_far += example[index]
    if result:
        print(so_far)
        print("****")
        so_far = ""
        
print(so_far)

Stocks were up as advancing issues outpaced declining issues on the NYSE by 1.5 to 1.
****
 Large- and small-cap stocks were both strong, while the S.&P.
****
 500 index gained 0.46% to finish at 2,457.59.
****
 Among individual stocks, the two top percentage gainers in the S.&P.
****
 500 were Incyte Corporation and Gilead Sciences Inc.
****



1. How many sentences are detected using your end of sentence classifier? **5**
2. Where did your end of sentence classifier make a mistake? **The classifier mistakenly classifies 'S.&P.' as the end of a sentence because it ends with ". "**

Task 2: Determining Sentiment
----

In [3]:
# we'll use nltk to access the reviews that we want to classify eventually
import nltk
import nltk.corpus as corpus

In [4]:
def load_word_list(filename):
    """
    Loads a lexicon from a plain text file in the format of one word per line.
    Parameters:
    filename (str): path to file

    Returns:
    list: list of words
    """
    with open(filename, 'r', encoding="utf-8") as f:
        # skip the header content
        for line in f:
            if line.strip() == "":
                break
        # read the rest of the lines into a list
        return [line.strip() for line in f]
    # otherwise return an empty list
    return []

In [5]:
# load in the positive and negative word lists here
# TODO: the paths to your negative/positive word files here
neg_lex = load_word_list("negative_words.txt")
pos_lex = load_word_list("positive_words.txt")

# TODO: How many words are in each list?
print(f"There are {len(neg_lex)} negative words")
print(f"There are {len(pos_lex)} positive words")


# TODO: Use python's list slicing to look at the first 10 elements in each list
print("First 10 negative words:")
print(neg_lex[:10])

print("First 10 positive words:")
print(pos_lex[:10])


There are 4783 negative words
There are 2006 positive words
First 10 negative words:
['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']
First 10 positive words:
['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']


In [6]:
# TODO: which words are in both the positive and the negative lists?
for neg_word in neg_lex:
    if neg_word in pos_lex:
        print(neg_word)

envious
enviously
enviousness


Now, we'll create our rule-based classifier! We have access to the word lists that you loaded and anything else you know about the world (reflect on how you as a human being can tell if a review is positive/negative). Your classifier need not be perfect, but it should be reasonable (don't just say everything is positive!).

In [7]:
def rule_based_classify(tokens, pos_lexicon, neg_lexicon, verbose = False):
    """
    This function classifies a given tokenized text as positive or negative
    based on the provided lexicons.
    Parameters:
    tokens (list) - list of strings tokenized words in the text to classify
    pos_lexicon (list) - list of strings words in the positive word lexicon
    neg_lexicon (list) - list of strings words in the negative word lexicon
    verbose (boolean) - flag indicating whether or not to print verbose (debugging) output. 
            Default value False.
    Returns:
    string "pos" if the list of tokens is positive overall, "neg" if they are negative overall.
    """
    score = 0
    for token in tokens:
        if token in pos_lexicon:
            if verbose:
                print(f"Found positive word: {token}")
            score += 1
        if token in neg_lexicon:
            if verbose:
                print(f"Found negative word: {token}")
            score -= 1
    if verbose:
        print(f"Score:", score)
    if score >= 0:
        return "pos"
    return "neg"

In [8]:
# now, we'll test out your classifier!
# Here are two example movie reviews.
nltk.download('movie_reviews')
movies = corpus.movie_reviews

# load in a single negative review
negative_toks = movies.words('neg/cv001_19502.txt')
# uncomment the text below to see the contents of the review
neg_text = " ".join(negative_toks)
# print(neg_text)

# load in a single positive review
positive_toks = movies.words('pos/cv992_11962.txt')
pos_text = " ".join(positive_toks)


# TODO:
# call your rule_based_classify on these example reviews.
print("Negative review classified as", rule_based_classify(neg_text.split(), pos_lex, neg_lex, False))
print("Positive review classified as", rule_based_classify(pos_text.split(), pos_lex, neg_lex, False))

# Does our classification function label them correctly? Why or why not?
# take a look at the contents of the reviews
# print(neg_text)
# print(pos_text)

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\ryan4\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Negative review classified as pos
Positive review classified as pos


1. What labels does your classifier assign these two reviews? __The classifier assigns them both to be positive__
2. Are these correct? __The negative review as labelled incorrectly, but the positive review was labelled correctly__

Task 3: How good is your sentiment classifier?
-----

Given the movies dataset from `nltk`, how many of the reviews does your classifier classify correctly?

We'll look at three different metrics: __accuracy__, __precision__, and __recall__.

__accuracy__: what you think of when you think of correctness.
$$ \frac{\texttt{number correct}}{\texttt{total number}}$$

Precision and recall require differentiated between the ways in which the classifier can be correct or incorrect. 

- __true positive__: an example whose gold label is positive and that the classifier labels as positive
- __true negative__: an example whose gold label is negative and that the classifier labels as negative
- __false positive__: an example whose gold label is negative and that the classifier labels as positive
- __false negative__: an example whose gold label is positive and that the classifier labels as negative

In [9]:
import random
# you can use numpy's random functionality if you'd like to
import numpy as np

In [10]:
# To see the available file ids, this is one way that we can access them.
# This will give you a list of neg/positive file ids.
print(len(movies.fileids('neg')))
# choose 100 random items without replacement from a list
print(random.sample(movies.fileids('neg'), 100))
print(len(movies.fileids('pos')))

1000
['neg/cv571_29292.txt', 'neg/cv701_15880.txt', 'neg/cv779_18989.txt', 'neg/cv205_9676.txt', 'neg/cv688_7884.txt', 'neg/cv378_21982.txt', 'neg/cv693_19147.txt', 'neg/cv531_26838.txt', 'neg/cv141_17179.txt', 'neg/cv848_10061.txt', 'neg/cv651_11120.txt', 'neg/cv633_29730.txt', 'neg/cv806_9405.txt', 'neg/cv650_15974.txt', 'neg/cv059_28723.txt', 'neg/cv287_17410.txt', 'neg/cv237_20635.txt', 'neg/cv209_28973.txt', 'neg/cv090_0049.txt', 'neg/cv652_15653.txt', 'neg/cv325_18330.txt', 'neg/cv879_16585.txt', 'neg/cv236_12427.txt', 'neg/cv830_5778.txt', 'neg/cv210_9557.txt', 'neg/cv614_11320.txt', 'neg/cv853_29119.txt', 'neg/cv756_23676.txt', 'neg/cv861_12809.txt', 'neg/cv615_15734.txt', 'neg/cv350_22139.txt', 'neg/cv424_9268.txt', 'neg/cv497_27086.txt', 'neg/cv730_10729.txt', 'neg/cv790_16202.txt', 'neg/cv835_20531.txt', 'neg/cv696_29619.txt', 'neg/cv659_21483.txt', 'neg/cv049_21917.txt', 'neg/cv052_29318.txt', 'neg/cv085_15286.txt', 'neg/cv604_23339.txt', 'neg/cv086_19488.txt', 'neg/cv479_5

In [11]:
# TODO:
# Write code that uses your classifier to classify 100 randomly chosen
# negative reviews and 100 randomly chosen positive reviews
# count the number of true positives, true negatives, false positives, and false negatives

# to get the tokens associated with a certain file id,
# tokens = movies.words(file_id)

# takes a long time to run if you loop over all fileids as opposed to just
# 100 randomly chosen ones
# make sure you don't classify the same review twice!
# (it takes us about 10 seconds to classify 200 reviews on a 2020 macbook air)

def classify_many_reviews(samples, review_type):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for sample in samples:
        tokens = movies.words(sample)
        result = rule_based_classify(tokens, pos_lex, neg_lex, False)
        if result == "pos":
            if review_type == "pos":
                tp += 1
            else:
                fp += 1
        elif result == "neg":
            if review_type == "neg":
                tn += 1
            else:
                fn += 1
    return tp, tn, fp, fn
        
neg_samples = random.sample(movies.fileids('neg'), 100)
pos_samples = random.sample(movies.fileids('pos'), 100)
    
_, tn, fp, _ = classify_many_reviews(neg_samples, "neg")
tp, _, _, fn = classify_many_reviews(pos_samples, "pos")
    
    
# TODO: print out the number of true positives, false positives,
# false negatives, and true negatives
print("True positives:", tp)
print("True negatives:", tn)
print("False positives:", fp)
print("False negatives:", fn)



True positives: 67
True negatives: 73
False positives: 27
False negatives: 33


Here are the equations for accuracy, precision, and recall in terms of what we've just been counting. $tp$ means true positive, $fp$ means false positive, $fn$ means false negative, and $tn$ means true negative.

$$ accuracy = \frac{tp + tn}{tp + fp + fn + tn}$$

$$ precision = \frac{tp}{tp + fp}$$

$$ recall = \frac{tp}{tp + fn}$$

You can think of precision as "how many of my positive guesses were correct?" and recall as "how many of the positive examples did I find?" 😄

In [12]:
# TODO: calculate and print accuracy
accuracy = (tp + tn)/(tp+fp+fn+tn)
print(accuracy)

0.7


In [13]:
# TODO: calculate and print precision
precision = tp/(tp+fp)
print(precision)

0.7127659574468085


In [14]:
# TODO: calculate and print recall
recall = tp/(tp+fn)
print(recall)

0.67


Task 4: n-gram math
----

Your final task in this lab is to do some math that will help you with your n-gram language model homework. Remember in HW 1 how you implemented a `count_list` function? Some of you were clever with how you implemented it, but let's look at a less clever implementation.

In [15]:
import time
from collections import Counter

def count_list(ls: list) -> dict:
    counts = {}
    for item in ls:
        # we're not going to be clever about counting here,
        # no conditionals, no sets, nothing
        counts[item] = ls.count(item)
    return counts

# see the difference between the following two items
example = [random.randint(0, 100) for i in range(2000)]
start = time.time()
count_list(example)
end = time.time()
print("That took:", end - start, "seconds!")

# this takes a very similar amount of time to count_dict from HW 1
start = time.time()
Counter(example)
end = time.time()
print("That took:", end - start, "seconds!")

That took: 0.1314988136291504 seconds!
That took: 0.0 seconds!


In [16]:
# TODO: put your create_ngrams (or make_ngrams) function here!
def make_ngrams(tokens: list, n: int) -> list:
    """Creates n-grams for the given token sequence.
    Args:
    tokens (list): a list of tokens as strings
    n (int): the length of n-grams to create

    Returns:
    list: list of tuples of strings, each tuple being one of the individual n-grams
    """
    ngrams = []
    if n > len(tokens):
        return ngrams
    for token in range(len(tokens)-n+1):
        temp = tuple(tokens[token:token+n])
        ngrams.append(temp)
    return ngrams

In [17]:
    # TODO: calculate the bigram score of the following sequence of tokens
    # for this example, we'll use a "vanilla" scoring technique
    # no Laplace smoothing, no unknown tokens
    training_data = ["<s>", "I", "love", "dogs", "</s>", "<s>", "I", "love", "cats", "</s>", "<s>", "I", "love", "dinosaurs", "</s>"]

    # TODO: call your create_ngrams function to get your bigrams

    bigrams = make_ngrams(training_data, 2)

    to_score = ["<s>", "I", "love", "cats", "</s>"]
    start = time.time()

    # BEGIN SCORING SECTION
    # start probability at one so that we can multiply the probability of
    # each subsequent next token with it
    total_prob = 1
    
    training_bigram_counts = Counter(bigrams)
    to_score_bigrams = make_ngrams(to_score, 2)
    
    #Chat-gpt revised this portion
    first_token_counts = {}
    for bigram in training_bigram_counts:
        if bigram[0] in first_token_counts:
            first_token_counts[bigram[0]] += training_bigram_counts[bigram]
        else:
            first_token_counts[bigram[0]] = training_bigram_counts[bigram]
    #End

    for i in range(1, len(to_score)):
        bigram = to_score_bigrams[i-1]
        if bigram not in training_bigram_counts:
            total_prob = 0
            pass
        numerator = training_bigram_counts[bigram]
        denom = first_token_counts[bigram[0]]
        this_prob = numerator / denom
        total_prob *= this_prob


    # END SCORING SECTION
    end = time.time()

    # print your final probability
    print("Final probability:", total_prob)
    print("That took", end - start, "seconds!")


Final probability: 0.3333333333333333
That took 0.0 seconds!


In [18]:
# Finally, pretend that we had a lot more data
training_data = ["<s>", "I", "love", "dogs", "</s>", "<s>", "I", "love", "cats", "</s>", "<s>", "I", "love", "dinosaurs", "</s>"]
# this is the amount of training data in the berp set
training_data = training_data * 3778

# TODO: call your create_ngrams function here


print("Number of training tokens:", len(training_data))
start = time.time()
# and what if we had 5000 sentences to score?
for example_num in range(3000):
    total_prob = 1
    
    training_bigram_counts = Counter(bigrams)
    to_score_bigrams = make_ngrams(to_score, 2)
    
    #Chat-gpt revised this portion
    first_token_counts = {}
    for bigram in training_bigram_counts:
        if bigram[0] in first_token_counts:
            first_token_counts[bigram[0]] += training_bigram_counts[bigram]
        else:
            first_token_counts[bigram[0]] = training_bigram_counts[bigram]
    #End

    for i in range(1, len(to_score)):
        bigram = to_score_bigrams[i-1]
        if bigram not in training_bigram_counts:
            total_prob = 0
            pass
        numerator = training_bigram_counts[bigram]
        denom = first_token_counts[bigram[0]]
        this_prob = numerator / denom
        total_prob *= this_prob 

end = time.time()
print("That took", end - start, "seconds!")

Number of training tokens: 56670
That took 0.06786227226257324 seconds!


What's the moral of the story? If you perform your counts at the same time you score, you'll be doing the same work over and over again which will result in a significantly slower model!

Make sure that you're gathering the counts that you need in `train` and only performing scoring calculations (as opposed to also counting things) in `score`.

This is particularly important when using larger data sets! (berp is not that big)